In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import os
import shutil

In [2]:
path2n2p2_result = '/home/y1u0d2/result/n2p2/SiO2/02/run5_feature_selected_each_bond'
path2mechanical_prop_files = '/home/y1u0d2/result/lammps/scripts/SiO2/elastic/template'

In [3]:
# 各epochのweight.*.outに対して、elastic constantを計算する
# result dirの中でelastic dirを作りその中に各epochのelastic fileを作る

In [4]:
path2elastic = os.path.join(path2n2p2_result, 'elastic')
if not os.path.exists(path2elastic):
    os.mkdir(path2elastic)

In [5]:
from ml_utils.util import remove_empty_from_array

def change_nnpdir(lines, path2nnpdir):
    for i, l in enumerate(lines):
        if 'variable nnpDir' in l:
            dir_l = remove_empty_from_array(l.split(' '))
            dir_l[-1] = f'"{path2nnpdir}"'
            lines[i] = ' '.join(dir_l)
            break
    return lines

In [6]:
weights_files_1 = glob(f'{path2n2p2_result}/weights.014.*')
weights_files_2 = glob(f'{path2n2p2_result}/weights.008.*')

weights_files_1.sort()
weights_files_2.sort()

for weights_f_1, weights_f_2 in zip(weights_files_1, weights_files_2):
    print(weights_f_1.split('/')[-1])
    if weights_f_1.split('/')[-1].split('.')[-2] != weights_f_2.split('/')[-1].split('.')[-2]:
        raise ValueError('epochが異なります')
    epoch = int(weights_f_1.split('/')[-1].split('.')[-2])
    path2epoch = os.path.join(path2elastic, f'e_{epoch}')

    # copy mechanical prop files
    if not os.path.exists(path2epoch): shutil.copytree(path2mechanical_prop_files, path2epoch)

    # copy n2p2 files
    shutil.copy(os.path.join(path2n2p2_result, 'input.nn'), path2epoch)
    shutil.copy(os.path.join(path2n2p2_result, 'scaling.data'), path2epoch)
    shutil.copy(weights_f_1, path2epoch)
    shutil.copy(weights_f_2, path2epoch)

    os.rename(os.path.join(path2epoch, os.path.basename(weights_f_1)), os.path.join(path2epoch, 'weights.014.data'))
    os.rename(os.path.join(path2epoch, os.path.basename(weights_f_2)), os.path.join(path2epoch, 'weights.008.data'))

    # change potential.mod
    with open(os.path.join(path2epoch, 'potential.mod'), mode='r') as f:
        lines = [s.strip() for s in f.readlines()]
    changed_lines = change_nnpdir(lines, path2epoch)
    with open(os.path.join(path2epoch, 'potential.mod'), mode='w') as f:
        f.write('\n'.join(changed_lines))
    
    # break

weights.014.000000.out
weights.014.000001.out
weights.014.000002.out
weights.014.000003.out
weights.014.000004.out
weights.014.000005.out
weights.014.000006.out
weights.014.000007.out
weights.014.000008.out
weights.014.000009.out
weights.014.000010.out
weights.014.000011.out
weights.014.000012.out
weights.014.000013.out
weights.014.000014.out
weights.014.000015.out
weights.014.000016.out
weights.014.000017.out
weights.014.000018.out
weights.014.000019.out
weights.014.000020.out
weights.014.000021.out
weights.014.000022.out
weights.014.000023.out
weights.014.000024.out
weights.014.000025.out
weights.014.000026.out
weights.014.000027.out
weights.014.000028.out
weights.014.000029.out
weights.014.000030.out
weights.014.000031.out
weights.014.000032.out
weights.014.000033.out
weights.014.000034.out
weights.014.000035.out
weights.014.000036.out
weights.014.000037.out
weights.014.000038.out
weights.014.000039.out
weights.014.000040.out
weights.014.000041.out
weights.014.000042.out
weights.014

In [7]:
# from analyze.lmp.mechanical_prop_each_epoch import make_elastic_dirs

# make_elastic_dirs(path2n2p2_result=path2n2p2_result, path2mechanical_prop_files=path2mechanical_prop_files)

In [8]:
# from glob import glob
# from analyze.lmp.mechanical_prop_each_epoch import make_elastic_dirs
# import subprocess
# import os


# path2n2p2_result = '/home/y1u0d2/result/n2p2/Si/84/02'
# path2mechanical_prop_files = '/home/y1u0d2/result/lammps/scripts/Si/elastic/template'

# make_elastic_dirs(path2n2p2_result=path2n2p2_result, path2mechanical_prop_files=path2mechanical_prop_files)

# elastic_dirs = glob(f'{path2n2p2_result}/elastic/e_*')
# for path in elastic_dirs:
#     try:
#         os.chdir(path)
#         process = subprocess.Popen(
#         f'mpirun -np 4 lmp -in in.elastic',
#         shell=True)
#         process.wait()
#     except:
#         print(path)